## Introduction to `robotoc` 
# 1: Getting Started

## インストール (Ubuntu)
1. OS が Ubuntu 22.04, 20.04, or 18.04であるか確認してください．（未確認ですが，Macでもインストール可能かもしれません．）
2. ビルドに必要なツールを以下でインストールしてください． 
```
sudo apt install build-essential
```
3. `Eigen3` を以下でインストールして下さい．
```
sudo apt install libeigen3-dev
```
4. [`Pinocchio`](https://github.com/stack-of-tasks/pinocchio) とそのビジュアライゼーションツール ([`gepeto-viewer-corba`](https://github.com/Gepetto/gepetto-viewer-corba))をインストールします． (詳細は [公式ドキュメンテーション](https://stack-of-tasks.github.io/pinocchio/download.html) を参照)
  - Ubuntu 22.04:
```
sudo apt install -qqy lsb-release gnupg2 curl -y
echo "deb [arch=amd64] http://robotpkg.openrobots.org/packages/debian/pub $(lsb_release -cs) robotpkg" | sudo tee /etc/apt/sources.list.d/robotpkg.list
curl http://robotpkg.openrobots.org/packages/debian/robotpkg.key | sudo apt-key add -
sudo apt update -y
sudo apt install -qqy robotpkg-py310-pinocchio -y
echo export PATH=/opt/openrobots/bin:$PATH >> ~/.bashrc
echo export PKG_CONFIG_PATH=/opt/openrobots/lib/pkgconfig:$PKG_CONFIG_PATH >> ~/.bashrc
echo export LD_LIBRARY_PATH=/opt/openrobots/lib:$LD_LIBRARY_PATH >> ~/.bashrc
echo export PYTHONPATH=/opt/openrobots/lib/python3.10/site-packages:$PYTHONPATH >> ~/.bashrc
echo export CMAKE_PREFIX_PATH=/opt/openrobots:$CMAKE_PREFIX_PATH >> ~/.bashrc
sudo apt update && sudo apt install robotpkg-py310-qt5-gepetto-viewer-corba -y
```
  - Ubuntu 20.04:
```
sudo apt install -qqy lsb-release gnupg2 curl -y
echo "deb [arch=amd64] http://robotpkg.openrobots.org/packages/debian/pub $(lsb_release -cs) robotpkg" | sudo tee /etc/apt/sources.list.d/robotpkg.list
curl http://robotpkg.openrobots.org/packages/debian/robotpkg.key | sudo apt-key add -
sudo apt update -y
sudo apt install -qqy robotpkg-py38-pinocchio -y
echo export PATH=/opt/openrobots/bin:$PATH >> ~/.bashrc
echo export PKG_CONFIG_PATH=/opt/openrobots/lib/pkgconfig:$PKG_CONFIG_PATH >> ~/.bashrc
echo export LD_LIBRARY_PATH=/opt/openrobots/lib:$LD_LIBRARY_PATH >> ~/.bashrc
echo export PYTHONPATH=/opt/openrobots/lib/python3.8/site-packages:$PYTHONPATH >> ~/.bashrc
echo export CMAKE_PREFIX_PATH=/opt/openrobots:$CMAKE_PREFIX_PATH >> ~/.bashrc
sudo apt update && sudo apt install robotpkg-py38-qt5-gepetto-viewer-corba -y
```
5. `robotoc` を以下でインストールします．
```
git clone https://github.com/mayataka/robotoc
cd robotoc
mkdir build && cd build
cmake .. -DCMAKE_BUILD_TYPE=Release -DOPTIMIZE_FOR_NATIVE=ON
sudo make install -j$(nproc)
```
  インストールできたら， Python の PATH を設定します．
  - Ubuntu 22.04:
```
echo export PYTHONPATH=/usr/local/lib/python3.10/site-packages:$PYTHONPATH >> ~/.bashrc
```
  - Ubuntu 20.04:
```
echo export PYTHONPATH=/usr/local/lib/python3.8/site-packages:$PYTHONPATH >> ~/.bashrc
```
6. `pybullet` と `meshcat` をインストールします．
```
pip3 install pybullet meshcat
```

## インストール (Mac)
以下は　Mac でのインストール方法ですがおそらくコンパイルエラーが起きます．   
Ubuntu の使用を強くお勧めします.   
1. `Pinocchio` をインストール
```
brew tap gepetto/homebrew-gepetto
brew install pinocchio
```
2. `g++` のフルパスを確認. (Mac のデフォルトCC++コンパイラは clang のため)
```
ls -l /usr/local/bin | grep g++
```
3. `robotoc` をインストール
```
git clone https://github.com/mayataka/robotoc
cd robotoc
mkdir build && cd build
cmake .. -DCMAKE_BUILD_TYPE=Release -DOPTIMIZE_FOR_NATIVE=ON -DCMAKE_CXX_COMPILER=FULL_PATH_TO_GPLUSPLUS
sudo make install -j$(nproc)
```
  Python の PATH をセット
```
echo export PYTHONPATH=/usr/local/lib/python3.9/site-packages:$PYTHONPATH >> ~/.zshrc
```
4. `pybullet` と `meshcat` をインストール
```
pip3 install pybullet meshcat
```

## Whole-body MPC
インストールがうまくいったかを確認するために， whole-body MPC をテストしてみます．   
細かい説明はこれ以降の notebook を参照してください．

### モジュールのインポート

In [ ]:
import robotoc
import numpy as np
from a1_simulator import A1Simulator

### ロボットの作成

In [ ]:
path_to_urdf = 'urdf/a1_description/urdf/a1.urdf'
contact_frames = ['FL_foot', 'RL_foot', 'FR_foot', 'RR_foot'] 
contact_types = [robotoc.ContactType.PointContact for i in contact_frames]
baumgarte_time_step = 0.05
robot = robotoc.Robot(path_to_urdf, robotoc.BaseJointType.FloatingBase, 
                      contact_frames, contact_types, baumgarte_time_step)

### Whole-body MPCの作成

In [ ]:
T = 0.5
N = 18
max_steps = 3
nthreads = 4
mpc = robotoc.MPCFlyingTrot(robot, T, N, max_steps, nthreads)

### Whole-body MPC に送るコマンド (`vcom_cmd`, `yaw_rate_cmd`) を設定

In [ ]:
# cmd_type = 'forward'
# cmd_type = 'backward'
# cmd_type = 'side'
# cmd_type = 'curve'
cmd_type = 'rotation'

if cmd_type == 'forward':
    step_length = np.array([0.2, 0.0, 0.0]) 
    step_yaw = 0.
elif cmd_type == 'backward':
    step_length = np.array([-0.1, 0.0, 0.0]) 
    step_yaw = 0.
elif cmd_type == 'side':
    step_length = np.array([0.0, 0.15, 0.0]) 
    step_yaw = 0.
elif cmd_type == 'curve':
    step_length = np.array([0.1, 0.0, 0.0]) 
    step_yaw = np.pi / 30.
elif cmd_type == 'rotation':
    step_length = np.array([0.0, 0.0, 0.0]) 
    step_yaw = np.pi / 20.
    
step_height = 0.1
stance_time = 0.15
flying_time = 0.1
swing_start_time = 0.5

vcom_cmd = 0.5 * step_length / (flying_time+stance_time)
yaw_rate_cmd = step_yaw / flying_time

### Foot step planner を作成

In [ ]:
planner = robotoc.FlyingTrotFootStepPlanner(robot)
raibert_gain = 0.9
planner.set_raibert_gait_pattern(vcom_cmd, yaw_rate_cmd, flying_time, stance_time, raibert_gain)
mpc.set_gait_pattern(planner, step_height, flying_time, stance_time, swing_start_time)

### シミュレーションの初期状態 (`q`, `v`) と初期時刻 (`t`) を設定

In [ ]:
q = np.array([0, 0, 0.3181, 0, 0, 0, 1, 
              0.0,  0.67, -1.3, 
              0.0,  0.67, -1.3, 
              0.0,  0.67, -1.3, 
              0.0,  0.67, -1.3])
v = np.zeros(robot.dimv())
t = 0.0

### MPCを初期化 (Newton反復を少し行います．)

In [ ]:
option_init = robotoc.SolverOptions()
option_init.max_iter = 10
mpc.init(t, q, v, option_init)

### MPC にオプション（サンプリング周期ごとの反復回数など）を設定 

In [ ]:
option_mpc = robotoc.SolverOptions()
option_mpc.max_iter = 2 # MPC iterations
mpc.set_solver_options(option_mpc)

### シミュレータを作成し，シミュレーションを行います．

In [ ]:
sim_time_step = 0.0025 # 400 Hz MPC
sim_start_time = 0.0
sim_end_time = 5.0

sim = A1Simulator(path_to_urdf, sim_time_step, sim_start_time, sim_end_time)

sim.set_camera(2.0, 45, -10, q[0:3]+np.array([0.1, 0.5, 0.]))
sim.run_simulation(mpc, q, v, feedback_delay=True, verbose=False, record=False, log=True, sim_name='mpc_flying_trot')
sim.disconnect()

### シミュレーション結果をプロット
(`scipy`, `matplotlib`, `seaborn` が必要です．例えば，`pip install scipy matplotlib seaborn`でインストールしてください．)

In [ ]:
q_log = np.genfromtxt(sim.q_log)
v_log = np.genfromtxt(sim.v_log)
t_log = np.genfromtxt(sim.t_log)
sim_steps = t_log.shape[0]

from scipy.spatial.transform import Rotation
vcom_log = []
wcom_log = []
vcom_cmd_log = []
yaw_rate_cmd_log = []
for i in range(sim_steps):
    R = Rotation.from_quat(q_log[i][3:7]).as_matrix()
    robot.forward_kinematics(q_log[i], v_log[i])
    vcom_log.append(R.T@robot.com_velocity()) # robot.com_velocity() is expressed in the world coordinate
    wcom_log.append(v_log[i][3:6])
    vcom_cmd_log.append(vcom_cmd)
    yaw_rate_cmd_log.append(yaw_rate_cmd)

%matplotlib inline
plot_mpc = robotoc.utils.PlotCoMVelocity()
plot_mpc.plot(t_log, vcom_log, wcom_log, vcom_cmd_log, yaw_rate_cmd_log, sim.sim_name)